# 🚀 Exercice 8 - Google Colab Ready

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Apierriecube/tp-rag-student-version/blob/main/exercice_8/Exercice_8_Evaluation.ipynb)

**Note**: Ce notebook peut s'exécuter sur Google Colab ou en local. Les cellules suivantes détectent automatiquement l'environnement.

## 🔧 Configuration pour Google Colab

Les cellules suivantes s'exécutent automatiquement sur Colab pour:
1. Installer les dépendances depuis GitHub
2. Configurer les clés API (secrets Colab)
3. Télécharger les fichiers nécessaires

In [1]:
# Détection de l'environnement
import sys
import os

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🌐 Environnement: Google Colab")
    print("📦 Installation des dépendances...")
else:
    print("💻 Environnement: Local")
    print("✓ Utilisation de l'environnement local")

💻 Environnement: Local
✓ Utilisation de l'environnement local


In [ ]:
# Installation des dépendances (Colab uniquement)
if IN_COLAB:
    # Cloner le repository ou télécharger les fichiers
    print("📥 Téléchargement des fichiers du projet...")
    
    # Option 1: Télécharger depuis GitHub (si disponible)
    # !git clone https://github.com/Apierriecube/tp-rag-student-version.git
    # %cd tp-rag-student-version/exercice_8
    
    # Option 2: Créer les fichiers directement (fallback)
    print("⚠️  Veuillez uploader manuellement:")
    print("   - helper_local.py")
    print("   - prompts_local.py")
    print("   (Utilisez l'icône 📁 dans le menu à gauche)")
    
    print("\n📦 Installation des dépendances (versions compatibles)...")
    # Fix transformers/peft compatibility issue
    !pip install -U transformers>=4.40.0 peft>=0.10.0
    !pip install -U \
        langchain>=0.1.20 \
        langchain-core \
        langchain-community \
        langchain-groq \
        langchain-huggingface \
        langchain-chroma \
        langchain-experimental \
        langchain-text-splitters \
        langgraph>=0.0.40 \
        trulens-eval>=0.30.0 \
        trulens-providers-litellm \
        chromadb>=0.4.22 \
        duckdb>=0.9.0 \
        pypdf>=3.17.0 \
        python-dotenv \
        pydantic \
        numpy pandas matplotlib seaborn
    
    print("✅ Installation terminée!")
    
    # Créer les dossiers nécessaires
    !mkdir -p data/arxiv
    !mkdir -p chroma_db
    
    print("\n📋 Prochaine étape: Configurer les clés API dans la cellule suivante")

In [ ]:
# Configuration des clés API
if IN_COLAB:
    from google.colab import userdata
    import os
    
    print("🔑 Configuration des clés API depuis Colab Secrets")
    print("\n💡 Pour ajouter des secrets dans Colab:")
    print("   1. Cliquez sur l'icône 🔑 (clé) dans le menu de gauche")
    print("   2. Ajoutez: GROQ_API_KEY = votre_clé_groq")
    print("   3. (Optionnel) HUGGINGFACE_API_KEY = votre_clé_hf")
    print("\n🌐 Obtenez votre clé Groq gratuite: https://console.groq.com/\n")
    
    try:
        # Récupérer les clés depuis Colab Secrets
        os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
        print("✅ GROQ_API_KEY configurée")
    except Exception as e:
        print(f"⚠️  GROQ_API_KEY non trouvée dans les secrets")
        print("   Ajoutez-la manuellement ci-dessous:")
        groq_key = input("Entrez votre GROQ_API_KEY: ")
        os.environ['GROQ_API_KEY'] = groq_key
    
    try:
        os.environ['HUGGINGFACE_API_KEY'] = userdata.get('HUGGINGFACE_API_KEY')
        print("✅ HUGGINGFACE_API_KEY configurée (optionnel)")
    except:
        print("ℹ️  HUGGINGFACE_API_KEY non trouvée (pas grave)")
    
    # TruLens config
    os.environ['TRULENS_OTEL_TRACING'] = '1'
    
    print("\n✅ Configuration terminée!")
else:
    # En local, utiliser .env
    from dotenv import load_dotenv
    load_dotenv()
    print("✅ Variables d'environnement chargées depuis .env")

In [ ]:
# Upload des fichiers nécessaires (Colab uniquement)
if IN_COLAB:
    from google.colab import files
    import os
    
    print("📁 Vérification des fichiers nécessaires...\n")
    
    # Vérifier si helper_local.py et prompts_local.py existent
    required_files = ['helper_local.py', 'prompts_local.py']
    missing_files = [f for f in required_files if not os.path.exists(f)]
    
    if missing_files:
        print(f"⚠️  Fichiers manquants: {', '.join(missing_files)}")
        print("\n📤 Veuillez uploader les fichiers:")
        print("   Option 1: Utilisez le menu 📁 à gauche pour glisser-déposer")
        print("   Option 2: Exécutez la cellule ci-dessous pour upload interactif\n")
        
        upload_choice = input("Voulez-vous uploader maintenant? (y/n): ")
        if upload_choice.lower() == 'y':
            print("\n📤 Sélectionnez helper_local.py et prompts_local.py:")
            uploaded = files.upload()
            for filename in uploaded.keys():
                print(f"✅ {filename} uploadé ({len(uploaded[filename])} bytes)")
    else:
        print("✅ Tous les fichiers nécessaires sont présents")
    
    # Optionnel: Upload de PDFs pour le RAG
    pdf_dir = 'data/arxiv'
    if not os.path.exists(pdf_dir) or len(os.listdir(pdf_dir)) == 0:
        print(f"\nℹ️  Aucun PDF trouvé dans {pdf_dir}/")
        print("   Le système utilisera des données de démonstration")
        print("   Pour utiliser vos PDFs, uploadez-les dans data/arxiv/")
else:
    print("✅ Mode local - fichiers déjà présents")

# Exercice 8 : Évaluation - Multi-Agent Data System Local

## Objectif

Modifier le notebook L6 du cours [Building and Evaluating Data Agents](https://www.deeplearning.ai/short-courses/building-and-evaluating-data-agents/) pour :

1. **Remplacer Snowflake Cortex par un RAG Local** avec indexation hiérarchique (ParentDocumentRetriever)
2. **Remplacer la partie SQL Snowflake par DuckDB local**
3. **Utiliser Groq** avec llama-3.1-8b-instant (gratuit)
4. **Évaluer avec TruLens** : RAG Triad + GPA (Goal-Plan-Action)
5. **Optimiser** avec inline evaluations et ajustement des prompts

## Architecture

```
┌─────────────┐
│   User      │
│   Query     │
└──────┬──────┘
       │
       ▼
┌─────────────┐      ┌──────────────────┐
│   Planner   │ ────▶│    Executor      │
└─────────────┘      └────────┬─────────┘
                              │
              ┌───────────────┼───────────────┐
              │               │               │
              ▼               ▼               ▼
    ┌─────────────────┐ ┌────────────┐ ┌──────────┐
    │ Local RAG       │ │  DuckDB    │ │  Chart   │
    │ Researcher      │ │ Researcher │ │Generator │
    │ (hierarchical)  │ │  (SQL)     │ │          │
    └─────────────────┘ └────────────┘ └──────────┘
              │               │               │
              └───────────────┼───────────────┘
                              │
                              ▼
                      ┌──────────────┐
                      │ Synthesizer  │
                      └──────────────┘
                              │
                              ▼
                      ┌──────────────┐
                      │   Answer     │
                      └──────────────┘
```

## Configuration et Installation

In [ ]:
# Installation des dépendances (LOCAL uniquement)
# Sur Colab, l'installation a déjà été faite plus haut

if not IN_COLAB:
    !pip install -q -r requirements.txt
    print("\n" + "="*80)
    print("Installation terminée !")
    print("="*80)
else:
    print("✓ Dépendances déjà installées sur Colab")

In [ ]:
import os

# Vérifier les clés API
print("\n📋 Configuration des API:")
print(f"  - GROQ_API_KEY: {'✓ Présent' if os.getenv('GROQ_API_KEY') else '✗ Manquant'}")
print(f"  - HUGGINGFACE_API_KEY: {'✓ Présent (optionnel)' if os.getenv('HUGGINGFACE_API_KEY') else '✗ Manquant (pas grave)'}")

# Vérifier que Groq est présent
if not os.getenv('GROQ_API_KEY'):
    print("\n⚠️  ATTENTION: GROQ_API_KEY manquant!")
    if IN_COLAB:
        print("   Retournez à la cellule de configuration des clés API")
    else:
        print("   Créez un fichier .env avec: GROQ_API_KEY=votre_clé")
        print("   Obtenez votre clé gratuite sur: https://console.groq.com/")
else:
    print("\n✅ Configuration API validée!")

## 1. Initialisation du système

Import des modules helper et prompts personnalisés

In [ ]:
# Import du module helper avec RAG local et DuckDB
from helper_local import (
    # State
    State,
    # Nodes
    planner_node,
    executor_node,
    local_rag_research_node,
    duckdb_research_node,
    chart_node,
    chart_summary_node,
    synthesizer_node,
    # Evaluations
    f_answer_relevance,
    f_context_relevance,
    f_groundedness,
    f_logical_consistency,
    f_execution_efficiency,
    f_plan_adherence,
    f_plan_quality,
    display_eval_reason,
)

from prompts_local import plan_prompt, executor_prompt

print("✓ Modules importés avec succès")

## 2. Ajout des inline evaluations

Nous ajoutons des évaluations inline pour les nœuds de recherche (RAG et DuckDB).
Cela permet d'obtenir du feedback en temps réel sur la qualité du contexte récupéré.

In [ ]:
from trulens.apps.langgraph.inline_evaluations import inline_evaluation
from trulens.core.otel.instrument import instrument
from trulens.otel.semconv.trace import SpanAttributes
from langgraph.types import Command
from typing import Literal
from langchain_core.messages import HumanMessage

# Import des agents et feedback
from helper_local import (
    local_rag_agent,
    duckdb_agent,
    f_context_relevance,
    State
)

# Nœud RAG avec inline evaluation
@inline_evaluation(f_context_relevance)
@instrument(
    span_type=SpanAttributes.SpanType.RETRIEVAL,
    attributes=lambda ret, exception, *args, **kwargs: {
        SpanAttributes.RETRIEVAL.QUERY_TEXT: args[0].get("agent_query", ""),
        SpanAttributes.RETRIEVAL.RETRIEVED_CONTEXTS: [
            ret.update["messages"][-1].content
        ] if hasattr(ret, "update") else "No retrieval",
    },
)
def local_rag_research_node_with_eval(state: State) -> Command[Literal["executor"]]:
    """Node for local RAG research with inline evaluation"""
    query = state.get("agent_query", state.get("user_query", ""))
    result = local_rag_agent.invoke({"messages": query})
    
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content,
        name="local_rag_researcher"
    )
    
    return Command(
        update={"messages": result["messages"]},
        goto="executor",
    )

# Nœud DuckDB avec inline evaluation
@inline_evaluation(f_context_relevance)
@instrument(
    span_type=SpanAttributes.SpanType.RETRIEVAL,
    attributes=lambda ret, exception, *args, **kwargs: {
        SpanAttributes.RETRIEVAL.QUERY_TEXT: args[0].get("agent_query", ""),
        SpanAttributes.RETRIEVAL.RETRIEVED_CONTEXTS: [
            ret.update["messages"][-1].content
        ] if hasattr(ret, "update") else "No query",
    },
)
def duckdb_research_node_with_eval(state: State) -> Command[Literal["executor"]]:
    """Node for DuckDB research with inline evaluation"""
    query = state.get("agent_query", state.get("user_query", ""))
    result = duckdb_agent.invoke({"messages": query})
    
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content,
        name="duckdb_researcher"
    )
    
    return Command(
        update={"messages": result["messages"]},
        goto="executor",
    )

print("✓ Inline evaluations configurées")

## 3. Amélioration des prompts de planification

Ajout de pre-conditions, post-conditions, et goals explicites pour chaque étape.

In [ ]:
# Le prompt est déjà amélioré dans prompts_local.py avec:
# - "pre_conditions": liste de préconditions
# - "post_conditions": liste de postconditions  
# - "goal": objectif explicite de l'étape

print("✓ Prompts améliorés avec sub-goals et conditions")
print("\nStructure d'une étape du plan:")
print("""{
  "agent": "local_rag_researcher",
  "action": "Search for information about RAG",
  "pre_conditions": ["User query received"],
  "post_conditions": ["Relevant documents retrieved"],
  "goal": "Find comprehensive information about RAG from local documents"
}""")

## 4. Construction du graphe multi-agent

In [ ]:
from langgraph.graph import START, StateGraph

# Créer le graphe
workflow = StateGraph(State)

# Ajouter les nœuds
workflow.add_node("planner", planner_node)
workflow.add_node("executor", executor_node)
workflow.add_node("local_rag_researcher", local_rag_research_node_with_eval)
workflow.add_node("duckdb_researcher", duckdb_research_node_with_eval)
workflow.add_node("chart_generator", chart_node)
workflow.add_node("chart_summarizer", chart_summary_node)
workflow.add_node("synthesizer", synthesizer_node)

# Point d'entrée
workflow.add_edge(START, "planner")

# Compiler le graphe
graph = workflow.compile()

print("✓ Graphe multi-agent construit")
print("\nNœuds du graphe:")
print("  - planner: Planifie les étapes")
print("  - executor: Exécute le plan")
print("  - local_rag_researcher: Recherche dans documents")
print("  - duckdb_researcher: Requêtes SQL structurées")
print("  - chart_generator: Génère des visualisations")
print("  - chart_summarizer: Résume les graphiques")
print("  - synthesizer: Synthèse finale")

## 5. Configuration de TruLens pour l'évaluation

In [ ]:
from trulens.core.session import TruSession
from trulens.core.database.connector.default import DefaultDBConnector

# Initialiser la connexion à la base de données TruLens
connector = DefaultDBConnector(database_url="sqlite:///trulens_exercice8.sqlite")
session = TruSession(connector=connector)

print("✓ Session TruLens initialisée")
print(f"  Base de données: trulens_exercice8.sqlite")

## 6. Enregistrement de l'agent avec TruLens

In [ ]:
from trulens.apps.langgraph import TruGraph

# Enregistrer l'agent avec tous les feedbacks
tru_recorder = TruGraph(
    graph,
    app_name="Local Data Agent",
    app_version="v1: Local RAG + DuckDB + Inline Evals + Sub-goals",
    feedbacks=[
        # RAG Triad
        f_answer_relevance,
        f_context_relevance,
        f_groundedness,
        # GPA (Goal-Plan-Action)
        f_logical_consistency,
        f_execution_efficiency,
        f_plan_adherence,
        f_plan_quality,
    ],
)

print("✓ Agent enregistré avec TruLens")
print("\nÉvaluations configurées:")
print("  RAG Triad:")
print("    - Answer Relevance")
print("    - Context Relevance")
print("    - Groundedness")
print("  GPA (Goal-Plan-Action):")
print("    - Logical Consistency")
print("    - Execution Efficiency")
print("    - Plan Adherence")
print("    - Plan Quality")

## 7. Tests du système

### Test 1: Requête sur documents (RAG)

In [ ]:
from langchain_core.messages import HumanMessage

# Test 1: Question sur les documents
query1 = "What is Retrieval Augmented Generation and what are its main advantages?"

print(f"Query 1: {query1}")
print("="*80)

with tru_recorder as recording:
    state1 = {
        "messages": [HumanMessage(content=query1)],
        "user_query": query1,
        "enabled_agents": [
            "local_rag_researcher",
            "duckdb_researcher",
            "chart_generator",
            "chart_summarizer",
            "synthesizer"
        ],
    }
    result1 = graph.invoke(state1)

print("\n" + "="*80)
print("RÉPONSE:")
print("="*80)
print(result1.get("final_answer", "No answer"))

### Test 2: Requête sur données structurées (DuckDB)

In [ ]:
# Test 2: Question sur les données de vente
query2 = "What are our top 3 deals by value? Show the company name and deal value."

print(f"Query 2: {query2}")
print("="*80)

with tru_recorder as recording:
    state2 = {
        "messages": [HumanMessage(content=query2)],
        "user_query": query2,
        "enabled_agents": [
            "local_rag_researcher",
            "duckdb_researcher",
            "chart_generator",
            "chart_summarizer",
            "synthesizer"
        ],
    }
    result2 = graph.invoke(state2)

print("\n" + "="*80)
print("RÉPONSE:")
print("="*80)
print(result2.get("final_answer", "No answer"))

### Test 3: Requête mixte avec visualisation

In [ ]:
# Test 3: Question avec graphique
query3 = "Show me a bar chart of deal values by product line for closed deals."

print(f"Query 3: {query3}")
print("="*80)

with tru_recorder as recording:
    state3 = {
        "messages": [HumanMessage(content=query3)],
        "user_query": query3,
        "enabled_agents": [
            "local_rag_researcher",
            "duckdb_researcher",
            "chart_generator",
            "chart_summarizer",
            "synthesizer"
        ],
    }
    result3 = graph.invoke(state3)

print("\n" + "="*80)
print("RÉPONSE:")
print("="*80)
print(result3.get("final_answer", "No answer"))

## 8. Analyse des résultats avec TruLens

In [ ]:
# Récupérer les enregistrements et feedbacks
records, feedback = session.get_records_and_feedback()

print(f"\nNombre d'exécutions enregistrées: {len(records)}")
print("\nDernières exécutions:")
print(records[["input", "output", "app_id"]].tail())

# Afficher les scores moyens
print("\n" + "="*80)
print("SCORES MOYENS DES ÉVALUATIONS")
print("="*80)

if not feedback.empty:
    # Grouper par type de feedback
    feedback_summary = feedback.groupby("name")["result"].agg(["mean", "count"])
    print(feedback_summary)

### Analyse détaillée d'une exécution

In [ ]:
# Analyser la dernière exécution
if len(records) > 0:
    last_record = records.iloc[-1]
    record_id = last_record["record_id"]
    
    print("\nAnalyse de la dernière exécution:")
    print("="*80)
    print(f"Input: {last_record['input'][:200]}...")
    print(f"\nOutput: {last_record['output'][:200]}...")
    
    # Récupérer les feedbacks pour cet enregistrement
    record_feedbacks = feedback[feedback["record_id"] == record_id]
    
    print("\nScores d'évaluation:")
    for _, fb in record_feedbacks.iterrows():
        print(f"  {fb['name']}: {fb['result']:.3f}")
        if pd.notna(fb.get('reason')):
            print(f"    Raison: {fb['reason'][:100]}...")

## 9. Lancement du dashboard TruLens

In [ ]:
from trulens.dashboard import run_dashboard

# Lancer le dashboard
port = 8006
print(f"Lancement du dashboard TruLens sur le port {port}...")
print(f"\nAccédez au dashboard à l'adresse: http://localhost:{port}")
print("\nUtilisez le dashboard pour:")
print("  - Comparer les différentes versions")
print("  - Analyser les traces d'exécution")
print("  - Voir les scores RAG Triad et GPA")
print("  - Identifier les points d'amélioration")

run_dashboard(port=port)

## 10. Section Optimisation

### Comparaison des stratégies de chunking

In [ ]:
# Test de comparaison entre retrieval standard et hiérarchique
from helper_local import standard_retriever, parent_retriever

test_query = "What are the main challenges in RAG systems?"

print("Comparaison des stratégies de retrieval")
print("="*80)
print(f"Query: {test_query}\n")

# Standard retrieval
print("1. STANDARD RETRIEVAL (chunks fixes)")
print("-" * 80)
standard_docs = standard_retriever.get_relevant_documents(test_query)
print(f"Nombre de documents: {len(standard_docs)}")
if standard_docs:
    print(f"Longueur moyenne: {sum(len(d.page_content) for d in standard_docs) / len(standard_docs):.0f} caractères")
    print(f"Premier résultat (extrait): {standard_docs[0].page_content[:200]}...")

# Hierarchical retrieval
print("\n2. HIERARCHICAL RETRIEVAL (ParentDocumentRetriever)")
print("-" * 80)
parent_docs = parent_retriever.get_relevant_documents(test_query)
print(f"Nombre de documents: {len(parent_docs)}")
if parent_docs:
    print(f"Longueur moyenne: {sum(len(d.page_content) for d in parent_docs) / len(parent_docs):.0f} caractères")
    print(f"Premier résultat (extrait): {parent_docs[0].page_content[:200]}...")

print("\n" + "="*80)
print("OBSERVATIONS:")
print("="*80)
print("""
- Le retrieval hiérarchique récupère des chunks plus petits pour la recherche
  mais renvoie le document parent complet pour plus de contexte
- Cela permet de:
  * Améliorer la précision de la recherche (chunks petits)
  * Fournir plus de contexte au LLM (parent plus grand)
  * Réduire le bruit dans les résultats
""")

### Ajustement des tailles de chunks

In [ ]:
# Test avec différentes tailles de chunks
from helper_local import create_parent_document_retriever

print("Test de différentes configurations de chunking")
print("="*80)

configs = [
    {"child_size": 200, "parent_size": 1000, "name": "Petits chunks"},
    {"child_size": 400, "parent_size": 2000, "name": "Chunks moyens (défaut)"},
    {"child_size": 800, "parent_size": 4000, "name": "Grands chunks"},
]

for config in configs:
    print(f"\n{config['name']}:")
    print(f"  Child chunk size: {config['child_size']}")
    print(f"  Parent chunk size: {config['parent_size']}")
    print("  Recommandé pour:")
    
    if config['child_size'] <= 300:
        print("    - Recherche très précise")
        print("    - Requêtes spécifiques")
        print("    - Mais: peut perdre du contexte")
    elif config['child_size'] >= 600:
        print("    - Questions larges")
        print("    - Besoin de contexte étendu")
        print("    - Mais: moins de précision")
    else:
        print("    - Usage général")
        print("    - Bon compromis précision/contexte")

print("\n" + "="*80)
print("RECOMMANDATIONS D'OPTIMISATION:")
print("="*80)
print("""
1. Utilisez TruLens Context Relevance pour évaluer la qualité
2. Si Context Relevance < 0.7: réduire child_chunk_size
3. Si Groundedness < 0.8: augmenter parent_chunk_size
4. Testez avec vos requêtes réelles et ajustez
5. Documentez les changements et leurs impacts
""")

## Conclusion et Prochaines Étapes

### Ce qui a été accompli

✅ **Remplacement de Snowflake Cortex par RAG Local**
- Indexation hiérarchique avec ParentDocumentRetriever
- Comparaison de 2 stratégies (standard vs hiérarchique)
- Tailles de chunks ajustables

✅ **Remplacement de SQL Snowflake par DuckDB**
- Base locale avec données de démonstration
- Requêtes SQL via agent dédié
- Schema structuré (deals table)

✅ **Utilisation de Groq (gratuit)**
- llama-3.1-8b-instant pour l'inférence
- Performances rapides
- Pas de coût

✅ **Évaluations TruLens complètes**
- RAG Triad: Context Relevance, Groundedness, Answer Relevance
- GPA: Plan Quality, Plan Adherence, Execution Efficiency, Logical Consistency
- Inline evaluations pour feedback temps réel

✅ **Optimisation**
- Prompts avec sub-goals et pré/post-conditions
- Instrumentation complète des nœuds
- Dashboard pour analyse visuelle

### Améliorations possibles

1. **Données réelles**: Remplacer les données de démo par vos vraies données
2. **Plus de documents**: Ajouter plus de PDFs dans data/arxiv
3. **Agents spécialisés**: Ajouter d'autres agents (web search, API calls, etc.)
4. **Fine-tuning**: Ajuster les prompts selon vos cas d'usage
5. **A/B Testing**: Comparer différentes configurations avec TruLens

### Ressources

- [Cours DeepLearning.AI](https://www.deeplearning.ai/short-courses/building-and-evaluating-data-agents/)
- [Documentation TruLens](https://www.trulens.org/)
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [Groq Documentation](https://groq.com/)
- [DuckDB Documentation](https://duckdb.org/)